In [ ]:
import os
from dotenv import load_dotenv
import requests
from huggingface_hub import InferenceClient

# Load environment variables from .env file
load_dotenv()

# Get token from .env file
hf_token = os.getenv('HUGGING_FACE_TOKEN')

if not hf_token:
    raise ValueError("HUGGING_FACE_TOKEN not found in .env file")

print("Token loaded successfully!")

In [ ]:

client = InferenceClient(api_key=hf_token)


In [ ]:
class PersonalAssistant:
    def __init__(self, client, role="You are a personal assistant that responds to a users input"):
        self.client = client
        self.system_role = role
        self.commands = ["generate books"]  # Add more commands as needed

    def execute_command(self,command):
        print(f"Executing command: {command}...")
        # Add logic for executing the specific command here

    def interact_with_model(self):
      """
      Continuously ask the user for input and interact with the model.
      """
      while True:
          # Get user input for the next question or query
          user_input = input("Ask a question (or type 'exit' to stop): ").strip()

          if user_input.lower() == 'exit': 
              print(self.speak(user_input))
              print("Exiting the program.")
              break

          #Check if user input is directly a command
          if user_input.lower() in (cmd.lower() for cmd in self.commands):
              self.execute_command(user_input.lower())
              continue

          # Pass the input to the speak function and get the model's response
          model_response = self.speak(user_input=user_input)

          # Check if the model's response matches a command
          if model_response.lower() in (cmd.lower() for cmd in self.commands):
              self.execute_command(model_response.lower())
              
          else:
              print(f"Model response: {model_response}")


    def speak(self, user_input="None"):
        """
        Generate a response based on the user input and predefined commands.
        """
        # Prepare the model input prompt based on user input
        if user_input.lower() == "none":
            prompt = "User has not said anything yet. Respond in a detailed and informative manner."
        elif user_input.lower() == 'exit':
            prompt = "User wishes to exit the program. Wish them farewell. Your response should only include the farewell message."
        else:
            prompt = (
                f"Case 1: If the user input: {user_input} matches or looks like one of these commands: {self.commands}, "
                "respond only with the command name. "
                f"Case 2: If the user input: {user_input} does not match any command in the list {self.commands}, "
                "acknowledge the user input and respond that you have a list of commands, then provide a numbered list of the commands. do not think out loud."
                f"Your response for case 1 should only include the command name"
                f"Your response for case 2 should only include a reply and Here is a list of avaliable commands. Followed by the list of commands"
            )

        # Create the messages for the model
        messages = [
            {"role": "system", "content": self.system_role},
            {"role": "user", "content": prompt},
        ]

        try:
            # Generate the response
            completion = self.client.chat.completions.create(
                model="Qwen/Qwen2.5-Coder-32B-Instruct",  # Replace with the correct model
                messages=messages,
                max_tokens=100,  # Adjust the token limit as needed
            )

            # Get and return the model's response
            return completion.choices[0].message["content"].strip()

        except Exception as e:
            print(f"Error occurred: {e}")
            return "An error occurred. Please try again."





In [ ]:
personal_assistant = PersonalAssistant(client)
personal_assistant.interact_with_model()